Imports - импортируем все необходимое

In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [36]:
X_full = pd.read_csv('giving_departures.csv', index_col='id')


In [37]:
X_full

,down_time_train,interval_giving,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time,route_id,sender_id,receiver_id,real_wagon_loading,real_train_weight_gross,real_train_wight_net,cargo_type_id,empty_types_id,down_time,axis_count
id,,,,,,,,,,,,,,,,,
24,0.583333,0.0,4.950000,-7.600000,2.40000,-2.050008,NaN,7506,109,3825,62,1635.0,0.0,1141,0,4.950000,248
32,2.002778,0.0,26.000000,-17.666667,NaN,NaN,NaN,7502,505,131,70,1657.9,0.0,1141,0,26.000000,280
33,2.666667,0.0,2.283333,2.966667,NaN,NaN,NaN,7503,132,131,71,1681.0,0.0,1141,0,2.283333,284
34,2.000000,0.0,6.533333,3.466667,NaN,0.900000,NaN,7503,82,131,71,1681.0,0.0,1141,0,6.533333,284
35,3.000000,0.0,3.650000,3.183333,NaN,NaN,NaN,7503,132,131,67,1583.0,0.0,1141,0,3.650000,268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5394,0.750000,0.0,1.500000,3.333333,NaN,NaN,0.433333,7492,131,505,70,6574.9,4890.5,5229,1,1.500000,280
5395,2.416667,0.0,5.733333,2.683333,NaN,NaN,NaN,7502,505,131,69,1647.0,0.0,1141,0,5.733333,276
5396,0.500000,0.0,2.350000,35.900000,5.15004,1.116672,4.266667,7500,485,82,53,4961.8,3708.0,102,1,2.350000,212


Data explorationData exploration

In [38]:
print(X_full.columns)

Index(['down_time_train', 'interval_giving', 'idle_train', 'travel_time',
       'crew_down_time', 'crew_down_time_plan', 'intermediate_down_time',
       'route_id', 'sender_id', 'receiver_id', 'real_wagon_loading',
       'real_train_weight_gross', 'real_train_wight_net', 'cargo_type_id',
       'empty_types_id', 'down_time', 'axis_count'],
      dtype='object')


In [39]:
X_full.sample(5)

,down_time_train,interval_giving,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time,route_id,sender_id,receiver_id,real_wagon_loading,real_train_weight_gross,real_train_wight_net,cargo_type_id,empty_types_id,down_time,axis_count
id,,,,,,,,,,,,,,,,,
26922,5.866667,0.0,2.316667,3.150000,NaN,NaN,NaN,7496,131,3925,67,6273.0,4682.1,5229,1,2.316667,268
17755,1.283333,0.0,3.766667,3.283333,NaN,NaN,NaN,7492,131,505,70,6553.5,4891.3,5229,1,3.766667,280
2183,2.416667,0.0,2.200000,3.050000,NaN,NaN,NaN,7502,505,131,70,1682.0,0.0,1141,0,2.200000,280
25981,1.516667,1.0,7.300000,3.516667,NaN,NaN,NaN,7497,241,82,67,6262.8,4677.0,5611,1,7.300000,268
7457,0.500000,0.0,1.833333,3.333333,NaN,NaN,NaN,7492,131,505,70,6555.7,4892.5,5229,1,1.833333,280


In [41]:
#X_full.describe(include='object')

In [42]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33246 entries, 24 to 5969
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   down_time_train          33246 non-null  float64
 1   interval_giving          33246 non-null  float64
 2   idle_train               33246 non-null  float64
 3   travel_time              33246 non-null  float64
 4   crew_down_time           11743 non-null  float64
 5   crew_down_time_plan      3834 non-null   float64
 6   intermediate_down_time   19841 non-null  float64
 7   route_id                 33246 non-null  int64  
 8   sender_id                33246 non-null  int64  
 9   receiver_id              33246 non-null  int64  
 10  real_wagon_loading       33246 non-null  int64  
 11  real_train_weight_gross  33246 non-null  float64
 12  real_train_wight_net     33246 non-null  float64
 13  cargo_type_id            33246 non-null  int64  
 14  empty_types_id           33

Data preparation - осуществляем базовые преобразования датасета

In [43]:
# Удаляем из датасета строки, в которых недостаточно данных для обучения.
X_full.dropna(axis=0, subset=['down_time_train'], inplace=True)
# Разбиваем датасет на цель (y) и входные данные (X)
y = X_full.down_time_train
X_full.drop(['down_time_train'], axis=1, inplace=True)

# Разбиваем датасет на train/val
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [45]:
# В наших данных есть нечисловые значения. Они затрудняют обработку
# поэтому мы оставим только те данные, где нечисловые значения имеют меньше 10 разных значений
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                    X_train_full[cname].dtype == "object"]

# и все числовые данные
numerical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].dtype in ['int64', 'float64']]

# все столбцы, в которых количество пропусков больше 10% X[cname].isnull().sum()

# оставим в датасете только те колонки, с которыми хотим работать дальше
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


Sanity check

In [46]:
print(len(X_train.columns), len(X_train_full.columns))
X_train.head()

16 16


,interval_giving,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time,route_id,sender_id,receiver_id,real_wagon_loading,real_train_weight_gross,real_train_wight_net,cargo_type_id,empty_types_id,down_time,axis_count
id,,,,,,,,,,,,,,,,
31502,0.0,14.800000,3.116667,NaN,NaN,0.433333,7504,82,241,70,1656.0,0.0,1141,0,14.800000,280
27793,0.0,3.316667,3.933333,NaN,NaN,0.933333,7502,505,131,69,1638.0,0.0,1141,0,3.316667,276
26657,0.0,2.316667,31.300000,6.016680,NaN,0.500000,7474,1465,82,53,4957.0,3704.5,102,1,2.316667,212
22819,0.0,5.216667,35.200000,10.283328,NaN,0.783333,7501,82,485,69,1642.0,0.0,1141,0,5.216667,276
16027,0.0,3.100000,3.016667,NaN,NaN,0.083333,7502,505,131,70,1659.0,0.0,1141,0,3.100000,280


#Pipeline

Baseline - базовый пайплайн, от которого мы будем отталкиваться в дальнейшем

###Data preparation (preprocessing)

In [47]:
# Инициализируем препроцессор для заполнения недостающих числовых значений
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Инициализируем препроцессор для заполнения недостающих НЕ числовых значений
# Добавляем шаг кодирования НЕ числовых значений в числовые
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Создаем общий препроцессор данных, соединив первые два
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ])

### Model

In [48]:
# Инициализируем модель
model = RandomForestRegressor(n_estimators=200, random_state=0, verbose=True)

In [49]:
# Создаем пайплайн из модели и препроцессора
classifier = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)],
                      verbose=True)

### Training

In [50]:
# Передаем обучающие данные в пайплайн, обучаем
classifier.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    7.9s


[Pipeline] ............. (step 2 of 2) Processing model, total=  30.9s


[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:   30.6s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   30.8s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['interval_giving',
                                                   'idle_train', 'travel_time',
                                                   'crew_down_time',
                                                   'crew_down_time_plan',
                                                   'intermediate_down_time',
                                                   'route_id', 'sender_id',
                                                   'receiver_id',
                                                   'real_wagon_loading',
                                                   'real_train_weight_gross',
                                                   'real_train_wight_net',
                                                   'cargo_type_id',
                                                   'empty_types_id',
                                                   'down_time', 'axis_count']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, random_state=0,
                                       verbose=True))],
         verbose=True)

### Model evaluation

In [28]:
# Передаем валидационные данные в пайплайн, получаем предсказания
preds = classifier.predict(X_valid)

# Оцениваем точность модели
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 1.255593328015313


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [29]:
y_valid.mean()

np.float64(1.8701498064855127)

In [30]:
(1 - 1.255593328015313/1.8701498064855127)*100

32.8613502693192

In [31]:
print('MSE:', mean_squared_error(y_valid, preds))

MSE: 6.9241569850247116


In [32]:
routes_to_check = 80
print(X_valid.iloc[routes_to_check].to_string())
print()
print(y_valid.iloc[routes_to_check])

facts_departures_id        17709.000000
transportation_sheet_id    64543.000000
interval_giving                1.000000
idle_train                     0.433333
travel_time                   34.883333
crew_down_time                 9.133344
crew_down_time_plan                 NaN
intermediate_down_time         1.933333

1.266667


In [33]:
classifier.predict(X_valid.iloc[routes_to_check:routes_to_check + 1])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([2.05266662])

In [34]:
r2_score(y_valid, preds)

0.07455610551753089